In [10]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [12]:
asins=[]

In [13]:
url='https://www.amazon.in/s?k=Phones&i=electronics&rh=n%3A1389432031%2Cp_89%3ARedmi&dc&qid=1623146584&rnid=3837712031&ref=sr_pg_1'

In [14]:
s = HTMLSession()
def getdata(url):
    r = s.get(url)
    soup=BeautifulSoup(r.text,'html.parser')
    items = r.html.find('div[data-asin]')
    for item in items:
        if item.attrs['data-asin'] != '':
            asins.append(item.attrs['data-asin'])
    return soup

def tonextpage(soup):
    page =soup.find('ul',{'class':'a-pagination'})
    if page is None:
        return 
    elif not page.find('li',{'class':'a-disabled a-last'}):
        url=f"https://www.amazon.in{page.find('li',{'class':'a-last'}).find('a')['href']}"
        return url
    else:
        return 
    
while True:
    soup=getdata(url)
    url=tonextpage(soup)
    print(url)
    if not url:
        break

https://www.amazon.in/s?k=Phones&i=electronics&rh=n%3A1389432031%2Cp_89%3ARedmi&dc&page=2&qid=1623313217&rnid=3837712031&ref=sr_pg_1
https://www.amazon.in/s?k=Phones&i=electronics&rh=n%3A1389432031%2Cp_89%3ARedmi&dc&page=3&qid=1623313218&rnid=3837712031&ref=sr_pg_2
https://www.amazon.in/s?k=Phones&i=electronics&rh=n%3A1389432031%2Cp_89%3ARedmi&dc&page=4&qid=1623313220&rnid=3837712031&ref=sr_pg_3
None


In [15]:
print(len(asins))

94


In [16]:
reviewlist = []
   
def get_soup(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

def get_reviews(soup):
    reviews = soup.find_all('div', {'data-hook': 'review'})
    try:
        for item in reviews:
            review = {
            'product': soup.title.text.replace('Amazon.in:Customer reviews:', '').strip(),
            'title': item.find('a', {'data-hook': 'review-title'}).text.strip(),
            'rating':  float(item.find('i', {'data-hook': 'review-star-rating'}).text.replace('out of 5 stars', '').strip()),
            'body': item.find('span', {'data-hook': 'review-body'}).text.strip(),
            }
            reviewlist.append(review)
    except:
        pass

y=0
for model in asins:    
    for x in range(1,5):
        soup = get_soup(f'https://www.amazon.in/product-reviews/{model}/ref=cm_cr_arp_d_paging_btm_next_2?pageNumber={x}')
        print(f'Getting page: {x}')
        get_reviews(soup)
        print(len(reviewlist))
        if not soup.find('li', {'class': 'a-disabled a-last'}):
            pass
        else:
            break
    if y==3:
        break
    y=y+1

Getting page: 1
10
Getting page: 2
20
Getting page: 3
30
Getting page: 4
40
Getting page: 1
40
Getting page: 2
50
Getting page: 3
50
Getting page: 4
60
Getting page: 1
70
Getting page: 2
80
Getting page: 3
80
Getting page: 4
90
Getting page: 1
100
Getting page: 2
110
Getting page: 3
120
Getting page: 4
120


In [17]:
reviewlist

[{'product': 'Redmi 9 (Sky Blue, 4GB RAM, 64GB Storage)| 5000 mAh| 2.3GHz Mediatek Helio G35 Octa core Processor',
  'title': 'DONT BUY THIS MOBILE .',
  'rating': 1.0,
  'body': 'Don’t buy this product .Image quality is below average comparing with the specification. Useless camera.And there is NO RETURN POLICY and replacing with same mobile is big headache .DON’T buy this mobile'},
 {'product': 'Redmi 9 (Sky Blue, 4GB RAM, 64GB Storage)| 5000 mAh| 2.3GHz Mediatek Helio G35 Octa core Processor',
  'title': 'Redmi 9 hhoo, 9, 9🤩, 9, changed mind',
  'rating': 4.0,
  'body': 'Let’s make one thing clear immediately: the Xiaomi Redmi 9 is a phone with flaws, and it has a lot of them. On the other hand, it is one of the most affordable phones available anywhere right now, and it\'s hard to find a better phone at this price point.The screen is of barely acceptable quality, the camera is disappointing, and the performance is middling at best, and yet, the Redmi 9 counters these with its super

In [18]:
df = pd.DataFrame(reviewlist)
df.to_csv('Amazon_review.csv')